Traducción al español de http://www.astropy.org/astropy-tutorials/Coordinates.html hecha por Germán Chaparro, junto con material adicional.

# Tutorial de Astropy en español

## Usando `astropy.coordinates` 

En esta parte del tutorial, exploraremos cómo el paquete `astropy.coordinates` junto con la funcionalidad general de astropy puede ser usado para planear observaciones y en general obtener información que requiera el uso de catálogos astronómicos grandes.

In [ ]:
import urllib
import IPython.display
import numpy as np

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

## Determinando lugares en el cielo con `coordinates`

Comencemos considerando un campo alrededor del pintoresco Grupo Compacto Hickson 7 (HCG7). Primero necesitamos obtener un objeto que represente las coordenadas del centro de este grupo.

En Astropy, el objeto más común para trabajar coordenadas es `SkyCoord`.  Un `SkyCoord` puede ser creado simplemente a partir de ángulos como se muestra más abajo. Es recomendable especificar el marco de referencia de sus coordenadas, aunque no es estrictamente necesario, ya que por defecto se usa el estándar ICRS de la IAU. Este estándar se puede entender como aproximadamente igual al sistema ecuatorial en el equinoxio de J2000, y se da en RA/DEC.

https://en.wikipedia.org/wiki/International_Celestial_Reference_System

In [ ]:
SkyCoord?

In [ ]:
hcg7_center = SkyCoord(9.81625*u.deg, 0.88806*u.deg, frame='icrs')
hcg7_center

`SkyCoord` también acepta coordenadas formateadas como `strings` ya sea como  `strings` separados para RA/DEC o como un `string` simple. Si las unidades no vienen en el `string` original, usted tiene que especificarlas.

In [ ]:
SkyCoord('23h39m15.9s', '0d53m17.016s', frame='icrs')

In [ ]:
SkyCoord('0:39:15.9 0:53:17.016', unit=(u.hour, u.deg), frame='icrs')

Si el objeto que le interesa está en [SESAME](http://cdsweb.u-strasbg.fr/cgi-bin/Sesame), usted puede hallarlo directamente usando el método de clase<sup>1</sup> `SkyCoord.from_name()`. Esto requiere que usted esté conectado a internet. Dado que ya lo definimos explícitamente, no es necesario ejecutar este paso si usted no está conectado a internet.

<sub> <sup>1</sup>Un objeto de clase es un constructor alternativo para un objeto `SkyCoord` object. Si usted llama la  `SkyCoord.from_name()` con un nombre, va a generar un nuevo objeto `SkyCoord`. Para más información sobre los objetos de clase y su uso, vea [esta página](https://julien.danjou.info/blog/2013/guide-python-static-class-abstract-methods).</sub>

In [ ]:
hcg7_center = SkyCoord.from_name('Hcg 7')
hcg7_center

In [ ]:
Black_Hole = SkyCoord.from_name('Sgr A*')
Black_Hole

El objeto que acabamos de crear nos da varias formas útiles de tener acceso a la información que almacena. En particular se usan los atributos ``ra`` y ``dec``, que son objetos [``Quantity``](http://docs.astropy.org/en/stable/units/index.html) de Astropy especializados (de hecho, pertenecen a una sub-clase llamada [``Angle``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html), cuyas subclases son [``Latitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Latitude.html) y [``Longitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Longitude.html)).  Estos objetos almacenan ángulos y muestran representaciones bonitas de ellos. También tiene atributos útiles para convertir fácilmente a unidades comunes de ángulo.

In [ ]:
type(hcg7_center.ra), type(hcg7_center.dec)

In [ ]:
hcg7_center.dec

In [ ]:
hcg7_center.ra

In [ ]:
hcg7_center.dec.hour

Ahora que tenemos un objeto `SkyCoord`, podemos usarlo para acceder a datos del [Sloan Digital Sky Survey](http://www.sdss.org/) (SDSS).  Empecemos intentando obtener una imagen usando la herramienta de recorte de imágenes de SDSS para asegurarnos que HGC7 está incluido en el catálogo SDSS y que tiene buena calidad de imagen.

Esto requiere estar conectado a internet, pero si falla, el archivo ``'HCG7_SDSS_cutout.jpg'`` se incluye en el repositorio.

In [ ]:
impix = 1024
imsize = 12*u.arcmin
cutoutbaseurl = 'http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx'
query_string = urllib.parse.urlencode(dict(ra=hcg7_center.ra.deg, 
                                     dec=hcg7_center.dec.deg, 
                                     width=impix, height=impix, 
                                     scale=imsize.to(u.arcsec).value/impix))
url = cutoutbaseurl + '?' + query_string

urllib.request.urlretrieve(url, 'HCG7_SDSS_cutout.jpg')

In [ ]:
urllib.parse.urlencode(dict(ra=hcg7_center.ra.deg, 
                                     dec=hcg7_center.dec.deg, 
                                     width=impix, height=impix, 
                                     scale=imsize.to(u.arcsec).value/impix))

Ahora miremos la imagen.

In [ ]:
IPython.display.Image('HCG7_SDSS_cutout.jpg')

¡Bonito!

### Ejercicios

Cree un `SkyCoord` de algún otro objeto astronómico de su interés. Usando una sola llamada/instrucción, obtenga la RA/Dec en la forma 'HH:MM:SS.S DD:MM:SS.S'.  Revise su respuesta contra [SIMBAD](http://simbad.u-strasbg.fr/simbad/), que le mostrará las coordenadas correctas para su objeto.

(Consejo: Puede valer la pena leer la documentación de `SkyCoord.to_string()`)

Ahora obtenga una imagen de su objeto del Digitized Sky Survey y descárguela y/o muéstrela en el notebook. Puntos extra si encuentra el truco (con una sola instrucción) para mostrar la imagen en el notebook *sin* descargar el archivo.

(Consejo: STScI tiene en sus archivos una [copia del DSS](https://archive.stsci.edu/dss/).  El patrón para la URL es ``http://archive.stsci.edu/cgi-bin/dss_search?f=GIF&ra=RA&dec=DEC``)

## Transformando entre sistemas de coordenadas y planeación de observaciones

Supongamos que usted quiere estudiar alguno de los objetos en este catálogo, y usted quiere saber cuando puede observarlo. Afortunadamente `astropy.coordinates` nos da esta información.

### Introduciendo transformaciones de coordenadas

Para enteder lo que viene a continuación, recomendamos leer el [resumen del esquema de coordenadas de astropy](http://astropy.readthedocs.org/en/latest/coordinates/index.html#overview-of-astropy-coordinates-concepts).  Lo importante es saber que todas las coordenadas en astropy están en "marcos de referencia" particulares, y podemos transformar el mismo objeto `SkyCoord` de un marco a otro.  Pdemos transformar nuestro centro de HCG7 de ICRS a coordenadas galácticas:

In [ ]:
hcg7_center.galactic

In [ ]:
hcg7_center

Lo anterior nos permite hacer una visualización rápida del cambio de coordenadas. También es posible usar el método `transform_to()` para hacer este cambio.

In [ ]:
from astropy.coordinates import Galactic
hcg7_center.transform_to(Galactic())

In [ ]:
hcg7_center.galactic.ra  # esto debe dar un mensaje de error, las coordenadas galácticas no se dan en RA/DEC

In [ ]:
hcg7_center.galactic.b

### Cambio a coordenadas AltAz

Para hacer observaciones, necesitamos hacer un cambio de coordenadas a un sistema basado en donde está el observador. Lo más común es usar coordenadas horizontales, o "AltAz". Necesitamos especificar desde donde queremos observar.

In [ ]:
from astropy.coordinates import EarthLocation
from astropy.time import Time

observing_location = EarthLocation(lat='4d', lon='-72d', height=2600*u.m)  # Bogotá
# Si se usa astropy v1.1 o más reciente, se puede reemplazar por la ubicación del observatorio:
#observing_location = EarthLocation.of_site('Kitt Peak')

observing_time = Time('2015-12-21 1:00')
#observing_time = Time.now()

Ahora usamos lo anterior para crear un objeto en el marco `AltAz`. Este marco tiene información sobre la atmósfera, que puede ser usado para corregir por refracción atmosférica.

In [ ]:
from astropy.coordinates import AltAz

aa = AltAz(location=observing_location, obstime=observing_time)
aa

Ahora transformamos nuestro `SkyCoord` en ICRS a `AltAz` para encontrar la ubicación en el cielo.

In [ ]:
hcg7_center.transform_to(aa)

Ejercicio: Determine la longitud galáctica en ángulo horario a la que apunta el cenit en este momento en Medellín.

## Unidades

In [ ]:
import astropy.units as u
import astropy.constants as const

In [ ]:
c=const.c

In [ ]:
freq=1420*u.MHz

In [ ]:
type(freq)

In [ ]:
c/freq

In [ ]:
(c/freq).to(u.m)

In [ ]:
(c/freq).decompose()

In [ ]:
print(const.G)

In [ ]:
c.cgs

In [ ]:
(1*u.au).si

# Archivos FITS

## Visualizando imágenes en archivos FITS

Imagen procesada del disco de escombros de Fomalhaut basada en imágenes tomadas por el Hubble Space Telescope. https://github.com/saint-germain/cazandoplanetas

In [ ]:
from astropy.io import fits

In [ ]:
fomalhaut=fits.open("https://github.com/saint-germain/cazandoplanetas/raw/66fb7ce82feb5e983e465b7cfd1a638d8bc3f038/Intermedio/PCA_FOMAL_full_centroids.fits")

In [ ]:
fomalhaut.info()

In [ ]:
hst_img=fomalhaut[0].data

In [ ]:
hst_img

In [ ]:
hst_img.max()

In [ ]:
plt.imshow(hst_img)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(hst_img*(hst_img>-0.0045)*(hst_img<0.004),cmap="gist_heat")
plt.scatter(1173,1205,s=1000,facecolors='none', edgecolors='b',lw=2)
plt.xlim(900,1250)
plt.ylim(1100,1400)

### Explorando un cubo de datos en un archivo FITS

Datos de espectros de CO galáctico (CO J = 1–0 line, vrest = 115.27 GHz) tomados por el radiotelescopio MINI (Crédito: Leonardo Bronfman, U. de Chile).

![](Screen Shot 2022-01-24 at 11.26.50 AM.png)

In [ ]:
# esta línea da un warning debido a que es un fits no estándar
cubo = fits.open("southgal_fixbadc.fits") #abrir objeto cubo de datos

In [ ]:
cubo.info()

In [ ]:
cubo[0].data

In [ ]:
cubo[0].header

In [ ]:
def values(h,j):
    # construye los ejes en velocidad, longitud y latitud con la información en el header
	N=h['NAXIS'+str(j)];
	val=np.zeros(N);
	for i in range(0,N):
		val[i] = (i+1-float(h['CRPIX'+str(j)]))*float(h['CDELT'+str(j)]) + float(h['CRVAL'+str(j)]);
	return val;

data 	= cubo[0].data #extraer matriz de datos
header	= cubo[0].header #extraer el header del archivo fits
#print header

#Estos seran los tres arreglos con los valores reales de los tres ejes del cubo
velocidad=values(header,1)
longitud=values(header,2)
latitud=values(header,3)

In [ ]:
# Escogemos algún valor arbitrario para longitud, latitud 
# para extraer el espectro para ese apuntamiento
i_l=-1
i_b=-1
T = data[i_b][i_l][:]

In [ ]:
lon=longitud[i_l]
lat=latitud[i_b]
plt.plot(velocidad,T)
plt.title(r'Espectro de la emisión de CO $J=1-0$ para ($l,b$)=(%i,%i)'%(lon,lat))

In [ ]:
maxeix=np.argmax(data,axis=2)
varr=np.array([velocidad[i] for i in maxeix.ravel()]).reshape(maxeix.shape)
plt.figure(figsize=(20,5))
plt.imshow(varr,cmap='bwr',extent=[longitud[0],longitud[-1],latitud[0],latitud[-1]])

También existe una librería llamada spectral_cube que puede procesar automáticamente este tipo de cubos de datos en formato fits (estandarizados): https://learn.astropy.org/tutorials/FITS-cubes.html

## Tablas (Votable) y WCS

Exploración de datos xml en formato Votable: el caso de las estrellas en el grupo móvil Lower Centaurus Crux (LCC).

Ir a SIMBAD, buscar "lcc", children objects, descargar Votable

Enlace:
http://simbad.u-strasbg.fr/simbad/sim-id?Ident=NAME%20LCC&NbIdent=query_hlinks&Coord=12%2019-57.1&parents=1&children=1938&submit=children&siblings=19&hlinksdisplay=h_all

In [ ]:
from astropy.io.votable import parse_single_table
votable = parse_single_table("lcc_simbad_votable.xml").to_table()

In [ ]:
votable

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot()
plt.scatter(votable['RA_d'],votable['DEC_d'])
plt.xlabel(r'RA')
plt.ylabel(r'Dec')

Las coordenadas mundiales (World Coordinates) sirven para ubicar una medida en un espacio de parámetros multidimensional. Un Sistema de Coordenadas Mundiales (WCS en inglés) especifica las coordenadas físicas, o mundiales (WC en inglés), que se adjuntan a cada píxel o spaxel de una imagen o matriz de $N$ dimensiones. Se ha desarrollado un elaborado conjunto de normas y convenciones para el formato FITS (Flexible Image Transport System) (Wells et al. 1981). Un ejemplo típico de WCS es la especificación de la Ascensión Recta (AR) y la Declinación (Dec) en el cielo asociada a una determinada ubicación de píxel o vóxel en una imagen celeste bidimensional (Greisen y Calabretta 2002; Calabretta y Greisen 2002).

Hay dos formas principales de inicializar un objeto WCS: con un diccionario de Python (o un objeto tipo diccionario, como la cabecera de un archivo FITS) o con listas de Python. En este ejemplo, inicializaremos un objeto astropy.wcs.WCS con dos dimensiones, como sería necesario para representar una imagen.

Adaptado y traducido de https://learn.astropy.org/tutorials/celestial_coords1.html

In [ ]:
from astropy.wcs import WCS
# world coordinate system https://learn.astropy.org/tutorials/celestial_coords1.html

In [ ]:
wcs_input_dict = {
    'CTYPE1': 'RA', 
    'CUNIT1': 'deg', 
    'CTYPE2': 'DEC', 
    'CUNIT2': 'deg'
}
wcs_helix = WCS(wcs_input_dict)

In [ ]:
wcs_helix

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs_helix)
plt.scatter(votable['RA_d'],votable['DEC_d'])
plt.xlabel(r'RA')
plt.ylabel(r'Dec')
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='black', ls='dotted')

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=wcs_helix)
plt.scatter(votable['RA_d'],votable['DEC_d'])
plt.xlabel(r'RA')
plt.ylabel(r'Dec')
overlay = ax.get_coords_overlay('galactic')
overlay.grid(color='black', ls='dotted')